### Configuration for Transmax streams gateway

In [45]:
import requests
import time
import json
import pandas
session = requests.Session()  # think of this as a new browser tab
session.proxies = {
    'http': 'localhost:3128',
    'https': 'localhost:3128'
}
#secrets


### Get Links - Returns information about the links that match the filter conditions.

In [222]:

url='https://api.dtmr.staging.data.streams.com.au'
headers_txt = {'Content-type': 'application/json','x-api-key': 'USoLc2B9De86v9QHy5ahcaXkXJd8Fq0a7MCwTI2V'}

ser_SIMSRecent  = url + "/Incidents/v1/sims"
payload_ser_aggDet = {'size': 5000}    


#-27.560914, 152.940195  Dara
#-27.555910, 152.940246  Centenary Hwy

ser_SIMSRecent  = url + "/gis/v1/links"
payload_ser_aggDet = {
        'size':500,
        'from': 1,
        'north': -27.5559,
        'east': 152.9402,
        'south':  -27.556,
        'west': 152.94033
         }   

response = session.get(ser_SIMSRecent,params=payload_ser_aggDet, headers=headers_txt, timeout=600)
response_json = response.json()
#print(response_json)

In [223]:
loop_streams_str = response_json['featureCollection']['features']
loop_list=[]

def change_cord_list(cord_list_old):
    '''Change lat long to long lat'''
    cord_list = []
    for cord in cord_list_old:
        lat_long = [cord[1], cord[0]]
        cord_list.append(lat_long)
    return cord_list

locations =[]

for site in loop_streams_str:
    ## only save sites with more than two coordinates
    if (len(site['geometry']['coordinates']) >1):
        #switch lat long
        new_cord_list =[]
        cord_list = site['geometry']['coordinates']
        new_cord_list = change_cord_list(cord_list)
        case = {'id': site['id'], 'geometry': new_cord_list, 'road':site['properties']['road'] }
        loop_list.append(case) 
        locations.append(new_cord_list)

In [227]:
from ipyleaflet import Map, Polyline, FullScreenControl

streams_line = Polyline(
    locations = [[
    locations,]],
    color = "black",
    fill_color= "black",
    weight=2)

m = Map(center = (-27.555910,152.940246), zoom =15)
m.add_layer(streams_line)
m.add_control(FullScreenControl())
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [126]:
response_json

{'featureCollection': {'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'id': '8484273:39:streams.metro.tmr.its',
    'geometry': {'type': 'LineString',
     'coordinates': [[152.940223, -27.556018],
      [152.940288, -27.55568],
      [152.940395850944, -27.5551318319793],
      [152.94053, -27.55445],
      [152.94068, -27.553661],
      [152.940735401219, -27.5533585745196],
      [152.940816709081, -27.5529147292519],
      [152.940816709084, -27.552914729236],
      [152.940816709084, -27.552914729236],
      [152.94085, -27.552733],
      [152.940998, -27.551941],
      [152.941138, -27.551235]]},
    'properties': {'description': 'Centenary Hwy NB between Sumners Nb On & Centenary Hwy Nb Dandenong Rd Ex',
     'startIntersectionId': '8484249:50:streams.metro.tmr.its',
     'endIntersectionId': '8484204:50:streams.metro.tmr.its',
     'length': 539,
     'freeFlowSpeed': 100,
     'road': 'Centenary Highway '}},
   {'type': 'Feature',
    'id': '8484287:39:str

###  Chosen road workds site: Centenary Highway - Darra
Works will commence in early-August 2019 and occur between 8pm and 5am, Sunday to Thursday, with the majority of work being on the road network. • Out of hours works will also be undertaken across the site to reduce impacts to the road network and ensure the safety of workers. • Speed will be reduced on Sumners Road to 40 kilometres per hour, and for both directions of the Centenary Motorway to 80 kilometres per hour in the active worksite areas. • Traffic control and signage will be in place as required. Please allow for extra time when planning your journey. • To ensure motorist and public safety, please follow all traffic and project related signage.

- Description 
On-road works will begin from August 2019 on the Sumners Road interchange Upgrade
- What is expected 
All lanes affected
All directions 
Delays expected (during active hours)
Changed traffic conditions
- Duration 
11 Aug 2019 to 31 Dec 2021

### HERE extract

In [267]:
incidentId ='Centenary Highway - Darra'
incidentCord = '-27.555910,152.940246'

#setup dataframe
dfcols = ['id','linkId','name','avSpeed','jamF','cords']
dfHere = pd.DataFrame(columns = dfcols)

#configure session request API
starttime = time.time()
urlsession = requests.session()
prox = "0.0001"# "20" #proximity in metres 

#payload
app_id=data['credentials'][0]['appid']
app_code=data['credentials'][0]['appcode']

#configure payload
url = "https://traffic.api.here.com/traffic/6.2/flow.json?app_id=" + app_id + "&app_code=" + app_code
url +="&prox="+incidentCord+",0.01&responseattributes=sh,fc&units=metric"

response = session.get(url)
response_json = response.json()

here_locations = [] # build list for polyline based on json coordinates
if response !="": #only process return values

    #process json return for output
    try:
        r=response_json#json.loads(response)   
        for el1 in r['RWS']:
            for el2 in el1['RW']:
                    for el3 in el2['FIS']: #Road level
                        for el4 in el3['FI']: #flow information extract here at link level
                            linRd = el4['TMC'].get('DE').replace("'","") #get rid of ' i.e "O'keefe Street" to "Okeefe Street"
                            linRd_pk =el4['TMC'].get('PC')
                            #print(linRd)
                            flowInfoSpeed = el4['CF'][0].get('SU') #Speed (based on UNITS) not capped by speed limit
                            flowInfoJam = el4['CF'][0].get('JF') # The number between 0.0 and 10.0 indicating the expected quality of travel. When there is a road closure, the Jam Factor will be 10. As the number approaches 10.0 the quality of travel is getting worse. -1.0 indicates that a Jam Factor could not be calculated
                            flowInfoCon =  el4['CF'][0].get('CN') #Confidence, an indication of how the speed was determined. -1.0 road closed. 1.0=100% 0.7-100% Historical Usually a value between .7 and 1.0
                            for el5 in el4['SHP']: #get shape file
                                cordStr = el5['value']
                                dfHere.loc[len(dfHere)] = [incidentId,linRd_pk,linRd, flowInfoSpeed,flowInfoJam,cordStr]
                                #create list for location
                                cord_string='[['+el5['value'][0]+']]'
                                cord_string = cord_string.replace(' -','],[-')
                                cord_string =eval(cord_string)
                                #here_locations.append(cord_string)  #variable for polyline generation
                                if (linRd == 'Sumners Road'):
                                #if (linRd_pk == 31862):
                                    here_locations.append(cord_string)  #variable for polyline generation

    except Exception as ex:
        print(str(response))
        raise ex       
    

In [270]:
#ouput from here geojson
from ipyleaflet import Map, Polyline, FullScreenControl

here_line = Polyline(
    locations = [[
    here_locations,]],
    color = "red",
    fill_color= "red",
    weight=2)

m = Map(center = (-27.555910,152.940246), zoom =13)
m.add_layer(here_line)
#m.add_layer(streams_line)

m.add_control(FullScreenControl())
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [271]:
# a=dfHere.groupby('name')
# a

group_names = dfHere.name.unique()
group_names

SyntaxError: invalid syntax (<ipython-input-271-6d217992882a>, line 3)

In [266]:
dfHere.loc[dfHere['linkId'] == 31862]

,id,linkId,name,avSpeed,jamF,cords
18,Centenary Highway - Darra,31862,Sumners Road,88.81,0.65232,"[-27.55609,152.94034 -27.55688,152.94018 ]"
19,Centenary Highway - Darra,31862,Sumners Road,88.81,0.65232,"[-27.55688,152.94018 -27.55767,152.94003 -27.5..."
20,Centenary Highway - Darra,31862,Sumners Road,88.81,0.65232,"[-27.55827,152.93995 -27.55857,152.93993 -27.5..."
21,Centenary Highway - Darra,31862,Sumners Road,88.81,0.65232,"[-27.55881,152.93993 -27.55884,152.93993 -27.5..."
22,Centenary Highway - Darra,31862,Sumners Road,88.81,0.65232,"[-27.55897,152.93994 -27.55913,152.93996 ]"
23,Centenary Highway - Darra,31862,Sumners Road,88.81,0.65232,"[-27.55913,152.93996 -27.55923,152.93997 ]"
24,Centenary Highway - Darra,31862,Sumners Road,88.81,0.65232,"[-27.55923,152.93997 -27.55935,152.93998 ]"
25,Centenary Highway - Darra,31862,Sumners Road,88.81,0.65232,"[-27.55935,152.93998 -27.55961,152.94001 -27.5..."
26,Centenary Highway - Darra,31862,Sumners Road,88.81,0.65232,"[-27.55176,152.94116 -27.55203,152.94111 ]"
27,Centenary Highway - Darra,31862,Sumners Road,88.81,0.65232,"[-27.55203,152.94111 -27.55516,152.94052 -27.5..."


In [245]:
dfHere[]

,id,linkId,name,avSpeed,jamF,cords
0,Centenary Highway - Darra,31860,Dandenong Road,82.08,1.71138,"[-27.54453,152.94256 -27.54496,152.94237 -27.5..."
1,Centenary Highway - Darra,31860,Dandenong Road,82.08,1.71138,"[-27.54656,152.94183 -27.54686,152.94176 ]"
2,Centenary Highway - Darra,31860,Dandenong Road,82.08,1.71138,"[-27.54686,152.94176 -27.54693,152.94174 -27.5..."
3,Centenary Highway - Darra,31860,Dandenong Road,82.08,1.71138,"[-27.54699,152.94173 -27.54709,152.94171 ]"
4,Centenary Highway - Darra,31860,Dandenong Road,82.08,1.71138,"[-27.54709,152.94171 -27.54745,152.94164 ]"
5,Centenary Highway - Darra,31860,Dandenong Road,82.08,1.71138,"[-27.54745,152.94164 -27.54768,152.9416 -27.54..."
6,Centenary Highway - Darra,31860,Dandenong Road,82.08,1.71138,"[-27.54787,152.94158 -27.54795,152.94157 ]"
7,Centenary Highway - Darra,31860,Dandenong Road,82.08,1.71138,"[-27.54795,152.94157 -27.54807,152.94156 ]"
8,Centenary Highway - Darra,31860,Dandenong Road,82.08,1.71138,"[-27.54807,152.94156 -27.54862,152.94151 ]"
9,Centenary Highway - Darra,31860,Dandenong Road,82.08,1.71138,"[-27.54862,152.94151 -27.54954,152.94147 ]"


#### Code below is not required.

In [156]:
## this code is for reverse geocoding 

#Set session proxies
import requests
import time
import json

session = requests.Session()  # think of this as a new browser tab
session.proxies = {
    'http': 'localhost:3128',
    'https': 'localhost:3128'
}
                
#location Centenary Highway - Darra
loc_lat=-27.555910
loc_long=152.940246

location=str(loc_lat)+','+str(loc_long)
prox='10'

#creditials load
data=pd.read_json("credentials.json")
app_id=data['credentials'][0]['appid']
app_code=data['credentials'][0]['appcode']
            
url = "https://reverse.geocoder.api.here.com/6.2/reversegeocode.json"
url +="?app_id="+app_id+"&app_code="+app_code
url +="&mode=retrieveAddresses&maxresults=1"
url +="&locationattributes=mapReference"
url +="&prox="+location+','+prox

response = session.get(url)
response_json = response.json()

here_link=response_json['Response']['View'][0]['Result'][0]['Location']['MapReference']['ReferenceId'] #PVids
here_link_street=response_json['Response']['View'][0]['Result'][0]['Location']['Address']['Street'] # name 3
pvid_x=response_json['Response']['View'][0]['Result'][0]['Location']['DisplayPosition']['Longitude']
pvid_y=response_json['Response']['View'][0]['Result'][0]['Location']['DisplayPosition']['Latitude']

#map results
from ipyleaflet import Map, Polyline, FullScreenControl,Marker
center = (pvid_y,pvid_x)# = pvid_x,pvid_y

m = Map(center = center, zoom =15)
marker = Marker(location=center, draggable=False)
m.add_layer(marker)

line = Polyline(
    locations = [[
    locations,]],
    color = "black",
    fill_color= "black",
    weight=2)
m.add_layer(line)
m.add_control(FullScreenControl())
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …